# Imports

In [2]:
import numpy as np
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import math
# import cv2
import os
import time
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import urllib.parse
import numpy as np
# Import dependencies
from urllib.parse import quote
from pprint import pprint
from googlesearch import search 
import random
from dateutil.parser import parse

In [3]:
def retry(ExceptionToCheck, tries=4, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    http://www.saltycrane.com/blog/2009/11/trying-out-retry-decorator-python/
    original from: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck:
                    msg = "%s, Retrying in %d seconds..." % (str(ExceptionToCheck), mdelay)
                    if logger:
                        #logger.exception(msg) # would print stack trace
                        logger.warning(msg)
                    else:
                        print(msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry  

## Dataset

In [4]:
df = pd.read_csv("../../data(fixed)/datasets/rotten_tomatoe_df.csv",index_col=0)
df = df.drop(df.columns[0], axis=1)

In [5]:
one,two,three,four,five,six,seven = df[:500],df[500:1000], df[1000:1500],df[1500:2000],df[2000:2500], df[2500:3000], df[3000:]

##### Selenium Arguments

In [6]:
# WINDOWS
# chromedriver_path = str("""C:/Users/Brian/Desktop/chromedriver.exe""")
# user_data_path = "C:/Users/Brian/AppData/Local/Google/Chrome/User Data/Thread_Five"

# MAC
chromedriver_path = "/Users/brianphelps/Desktop/chromedriver"
user_data_path = "/Users/brianphelps/Library/Application Support/Google/Chrome/Thread_SWOOPS"

# OPTIONS
options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("disable-dev-shm-usage")
options.add_argument("headless")
# options.add_argument(f"user-data-dir={user_data_path}")
# driver = webdriver.Chrome(f"{chromedriver_path}", chrome_options=options)
# driver.get('https://pro.imdb.com')

In [25]:
class RottenTomatoeSpider(object):
  def __init__(self, df):
    """Crawl Rotten Tomatoe search results

    This class is used to crawl Rotten Tomatoe's reviews using Selenium and BeautifulSoup.
    """
    super().__init__()
    chromedriver_path = "/Users/brianphelps/Desktop/chromedriver"
    user_data_path = "/Users/brianphelps/Library/Application Support/Google/Chrome/Thread_SWOOPS"

    # OPTIONS
    options = webdriver.ChromeOptions()
    options.add_argument("no-sandbox")
    options.add_argument("disable-dev-shm-usage")
    options.add_argument("headless")
#     options.add_argument(f"user-data-dir={user_data_path}")
    
    self.options = options
    self.chromedriver_path = chromedriver_path
    self.data = df
    self.review_dataframe = pd.DataFrame()

  def critic_reviews(self, thread_name):
    """Get the web page's source code

    Args:
        thread_name (str): Filename of thread to save

    Returns:
        dataframe: Critic review results
    """
    counter=0
    for row in self.data.itertuples():
      counter+=1
      profile = row.profile
      url = row.url
      name = row.name
      year = int(row.Year)
      '''
          GET REVIEWS
      '''

      driver = webdriver.Chrome(f"{self.chromedriver_path}", chrome_options=self.options)

      @retry((ConnectionResetError,TimeoutException,ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
      def driver_reviews(url):
        driver.get(f"{url}")
      driver_reviews(url+'/reviews')

      src = driver.page_source
      parser = BeautifulSoup(src, 'lxml')
      base_url = 'https://www.rottentomatoes.com/'

      try:
        num_pages = int(parser.find('span', class_='pageInfo').text.replace("Page 1 of ", "").strip())
      except:
        num_pages = 1


      review_count=0
      for i in range(num_pages):
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')
        try:
          next_link_one = parser.find('span', class_='pageInfo').parent.find_all('a')[1]['href']
          next_link = base_url+next_link_one
        except:
          next_link_one = 'nan'
          next_link = np.nan

        try:
          reviews_container = parser.find('div', class_='review_table')
          review_containers = reviews_container.find_all('div', class_='review_table_row')
        except:
          driver.quit()
          break

        movie_reviews = pd.DataFrame()

        for review in review_containers:
          # FIND SENTIMENT
          fresh = review.find('div', class_='fresh')
          if fresh:
            sentiment = 'fresh'
          else:
            sentiment = 'rotten'
              
          # TOP CRITIC BOOLEAN
          top = review.find('span', class_='glyphicon-star')
          if top:
            top_critic = True
          else:
            top_critic = False

          try:
            date = review.find('div', class_='review-date').text.strip()
          except:
            date = np.nan
            
          try:
            the_review = review.find('div', class_='the_review').text.strip()
          except:
            the_review = np.nan

          temp = pd.DataFrame({
            'date': [date],
            'review': [the_review],
            'sentiment': [sentiment],
            'movie': [name],
            'profile': [profile],
            'top_critic': [top_critic]
          })
          review_count+=1
          self.review_dataframe = self.review_dataframe.append(temp, ignore_index=True)
          print(Fore.GREEN + name)
          print(Fore.GREEN + f"Reviews: {review_count}")
          print(Fore.GREEN + f"{counter}/{len(self.data)}")
          clear_output(wait = True)
          
          
          
#         self.review_dataframe.to_csv(f"./rotten_tomatoe_critic_reviews_{thread_name}.csv", index=False)
        if next_link_one not in ["#","nan"] and type(next_link)==str:
          driver_reviews(next_link)
        else:
          driver.quit()
          
    return self.review_dataframe

  def audience_reviews(self, thread):
    """Search Rotten Tomatoe Audience Reviews

    Returns:
        dataframe: Audience Reviews
    """

    counter=0
    for row in self.data.itertuples():
      counter+=1
      profile = row.profile
      url = row.url
      name = row.name
      year = int(row.Year)
      '''
          GET REVIEWS
      '''

      driver = webdriver.Chrome(f"{self.chromedriver_path}", chrome_options=self.options)

      @retry((ConnectionResetError,TimeoutException,ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
      def driver_reviews(url):
        driver.get(f"{url}")
      driver_reviews(url+'/reviews?type=user')

      src = driver.page_source
      parser = BeautifulSoup(src, 'lxml')
      base_url = 'https://www.rottentomatoes.com/'


      current_movie_reviews = pd.DataFrame()
      review_count=0

      next_page_btn = True

      while next_page_btn == True:

        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')
        for button in parser.find_all("button", class_='hide'): 
          #delete all hidden buttons
          button.decompose()

        try:
          #Delete Element From DOM
          #PYTHON
          for button in parser.find_all("button", class_='hide'): 
            #delete all hidden buttons
            button.decompose()
          #JS
          driver.execute_script("""
            var elements = document.getElementsByClassName('prev-next-paging__button-right hide');
            for(i=0;i<elements.length;i++){
              elements[i].parentNode.removeChild(elements[i]);
            }
          """)
          next_link = parser.find('button', class_='prev-next-paging__button-right')
        except:
          next_link = 'nan'
          next_page_btn = False

        try:
          reviews_container = parser.find('ul', class_='audience-reviews')
          review_containers = reviews_container.find_all('li', class_='audience-reviews__item')
        except:
          driver.quit()
          break

        for review in review_containers:

          full = len(review.find_all('span', class_='star-display__filled'))
          half = len(review.find_all('span', class_='star-display__half'))
          sentiment = 0
          if full:
            for i in range(full):
              sentiment+=1
          if half:
            for i in range(half):
              sentiment+=.5


          try:
            date = review.find('span', class_='audience-reviews__duration').text.strip()
          except:
            date = np.nan
          try:
            the_review = review.find('p').text.strip()
          except:
            the_review = np.nan

          temp = pd.DataFrame({
            'date': [date],
            'review': [the_review],
            'sentiment': [sentiment],
            'movie': [name],
            'profile': [profile]
          })
          review_count+=1

          current_movie_reviews = current_movie_reviews.append(temp, ignore_index=True).drop_duplicates()
          print(Fore.GREEN + name)
          print(Fore.GREEN + f"Reviews: {review_count}")
          print(Fore.GREEN + f"{counter}/{len(self.data)}")
          clear_output(wait = True)
        if next_link!='nan':
          try:
            driver.execute_script("""
              var element = document.getElementsByClassName('prev-next-paging__button-right')[0];
              if(element.classList.contains('hide')){

              }else{
                element.click()
              }
            """)
            time.sleep(1)
          except:
            next_page_btn = False
        if self.review_dataframe.shape[0] > 250000:
          self.review_dataframe.to_csv(f"{thread}_{counter}.csv")
          self.review_dataframe = pd.DataFrame()
        self.review_dataframe = self.review_dataframe.append(current_movie_reviews, ignore_index=True).drop_duplicates()
        print(Fore.GREEN + name)
        print(Fore.GREEN + f"Reviews: {len(self.review_dataframe)}")
        print(Fore.GREEN + f"{counter}/{len(self.data)}")
        clear_output(wait = True)
      driver.quit()
    return self.review_dataframe

In [59]:
len(spider.review_dataframe.profile.unique())

7

In [60]:
saved_dataframe_one = spider.review_dataframe.copy()

In [61]:
one=one.reset_index(drop=True)
one = one.iloc[6:]
spider = RottenTomatoeSpider(one)
one_reviews = spider.audience_reviews('one')

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=86.0.4240.183)
